## Arc layer



In [1]:
"""
ArcLayer
========

Map of commutes to work within a segment of downtown San Francisco using a deck.gl ArcLayer.

Green indicates a start point, and red indicates the destination.

The data is collected by the US Census Bureau and viewable in the 2017 LODES data set: https://lehd.ces.census.gov/data/
"""

import pydeck
import pandas as pd

with open('../../../mapbox_key.txt', 'r') as f:
    mapbox_key = f.readline()
    print('Mapbox key =', mapbox_key)

DATA_URL = "https://raw.githubusercontent.com/ajduberstein/sf_public_data/master/bay_area_commute_routes.csv"
DOWNTOWN_BOUNDING_BOX = [
    -122.43135291617365,
    37.766492914983864,
    -122.38706428091974,
    37.80583561830737,
]


def in_bounding_box(point):
    lng, lat = point
    in_lng_bounds = DOWNTOWN_BOUNDING_BOX[0] <= lng <= DOWNTOWN_BOUNDING_BOX[2]
    in_lat_bounds = DOWNTOWN_BOUNDING_BOX[1] <= lat <= DOWNTOWN_BOUNDING_BOX[3]
    return in_lng_bounds and in_lat_bounds


df = pd.read_csv(DATA_URL)
# Filter to bounding box
df = df[df[["lng_w", "lat_w"]].apply(lambda row: in_bounding_box(row), axis=1)]

GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

Mapbox key = pk.eyJ1IjoiZWp3aWxsZW1zZSIsImEiOiJjazhxMG1wYTkwMGMwM3Jwa29qcTduYjBzIn0.Ak0jgr96rC-W5WB1jlmkyw


In [2]:
df

,S000,w_geocode,h_geocode,lat_w,lng_w,geoid10,lat_h,lng_h
1,2,60750615001027,60014004001009,37.788996,-122.399133,60014004001009,37.849728,-122.253480
7,2,60750117002022,60014011003009,37.787461,-122.404334,60014011003009,37.825330,-122.263653
8,5,60750176014001,60014011003009,37.776614,-122.416391,60014011003009,37.825330,-122.263653
9,2,60750615001008,60014011003009,37.791575,-122.393733,60014011003009,37.825330,-122.263653
10,2,60750615001016,60014011003009,37.789900,-122.399964,60014011003009,37.825330,-122.263653
...,...,...,...,...,...,...,...,...
189551,2,60750176014001,60855105003022,37.776614,-122.416391,60855105003022,37.386371,-122.121742
189564,2,60750177002034,60133560024022,37.767805,-122.414433,60133560024022,37.965198,-122.132659
189566,2,60750177002001,60816102014008,37.774430,-122.418313,60816102014008,37.485234,-122.222085
189568,2,60750177002001,60816138003051,37.774430,-122.418313,60816138003051,37.323417,-122.238966


In [4]:
arc_layer = pydeck.Layer(
    "ArcLayer",
    data=df,
    get_width="S000 * 2",
    get_source_position=["lng_h", "lat_h"],
    get_target_position=["lng_w", "lat_w"],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pydeck.ViewState(latitude=37.7576171, longitude=-122.5776844, bearing=45, pitch=50, zoom=8,)

In [7]:
TOOLTIP_TEXT = {"html": "{S000} jobs <br /> Home of commuter in red; work location in green"}
r = pydeck.Deck(arc_layer, 
                initial_view_state=view_state, 
                tooltip=TOOLTIP_TEXT, 
                map_style="mapbox://styles/mapbox/satellite-v9",
                mapbox_key=mapbox_key,
               height=1000)
r.show()

DeckGLWidget(height=1000, json_input='{"initialViewState": {"bearing": 45, "latitude": 37.7576171, "longitude"…